# Scope of jupyter notebook
1. Data preperation steps
2. Fitting the model
3. Comparing the model performance with subset of data

In [1]:
import keras, os, pickle, ast
# import implicit
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from IPython.display import SVG
from keras.models import model_from_json
from keras.optimizers import Adam
from keras.layers import Dense,Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from keras.callbacks import ReduceLROnPlateau, History
from keras.regularizers import l1,l2
import seaborn as sns
sns.set()


Using TensorFlow backend.


In [ ]:
rootPath = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

## Synthetic Data
1. We work with 3000 training examples with this neural network
2. Assuming 10% noise in data, we work with 2700 reliable datasets, 300 noise datsets (random labels)
3. Breaking down the 2700 training examples

#### we break into high, middle and low tier ratings with a few simple rules in place to guide the model to learn
1. high - high happy probability, moderate-low angry/disgusted, handraised 1, sleep 0, headgaze 2
2. mid - moderate emotions probability, handraise random, sleep 0, headgaze 1-2
3. low - low happy probability, moderate-high angry/disgusted, handraised 0, sleep 0/1, headgaze 0,1
4. noise - all are noise
5. engagement - we will take softmax of range 1 to 5, 4-5 being most engaged, 2-3 being mildly engaged, 1 least engaged

In [3]:
df = pd.read_excel('EngagementTest4.xlsx')

#remove first column from importing excel
df = df.drop('Unnamed: 0', axis = 1)

#shuffle dataframe for randomness in splitting
df = df.sample(frac = 1)

In [4]:
df.head()

,Happy,Angry,Disgusted,HandRaised,Sleep,HeadGaze,Engagement
2452,0.094595,0.531981,0.373424,0,1,1,1
656,0.522444,0.313164,0.164393,1,0,2,4
2405,0.102863,0.393491,0.503646,0,1,0,1
2686,0.051951,0.088931,0.859119,0,0,0,1
2494,0.084931,0.334487,0.580582,0,1,1,1


In [5]:
#changing values to numpy array
X = df.iloc[:,:6].to_numpy()
Y = df.iloc[:,6:].to_numpy()

print(X.shape)
print(Y.shape)

(3000, 6)
(3000, 1)


In [7]:
#normalizing the data, will help with different ranges of values for different features
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
scaled_df = pd.DataFrame(X)

scaled_df.head()

,0,1,2,3,4,5
0,0.096125,0.540757,0.382505,0.0,1.0,0.5
1,0.531764,0.318293,0.168283,1.0,0.0,1.0
2,0.104543,0.399959,0.515962,0.0,1.0,0.0
3,0.052703,0.090324,0.880263,0.0,0.0,0.0
4,0.086285,0.339972,0.594808,0.0,1.0,0.5


In [8]:
#apply one hot encoding to Y values
ohe = OneHotEncoder()
Y = ohe.fit_transform(Y).toarray()

C:\Users\Gabriel Sze\Anaconda3\envs\tensor\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
Y

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

In [10]:
#Split the dataset into x and y
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(2400, 6)
(600, 6)
(2400, 5)
(600, 5)


In [11]:
#Building the model, to train this we will build a simple 3 layer NN

def create_model():
    n_cols = X_train.shape[1]
    
    #batch normalization layers need to be added before relu activation
    model = keras.Sequential()
    model.add(Dense(50, activity_regularizer = l2(0.00001), input_shape = (n_cols,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(200))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(200))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
              
    model.add(Dense(200))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(200))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(5, activation = 'softmax'))
    
    return model

In [12]:
#training the model
#test if model has any serious loss/accuracy problems
model = create_model()
model.compile(optimizer = Adam(lr = 0.000005), loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
#adding callbacks
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 50, min_lr = 1e-7, verbose = 1)

history = History()

history = model.fit(X_train, Y_train,
                    validation_split = 0.2,
                    epochs = 2000,
                    batch_size = 64,
                    verbose = 2,
                    callbacks = [reduce_lr, history])


Train on 1920 samples, validate on 480 samples
Epoch 1/2000
 - 2s - loss: 2.2109 - categorical_accuracy: 0.2120 - val_loss: 1.6069 - val_categorical_accuracy: 0.1708
Epoch 2/2000
 - 0s - loss: 2.2224 - categorical_accuracy: 0.2005 - val_loss: 1.6126 - val_categorical_accuracy: 0.1708
Epoch 3/2000
 - 0s - loss: 2.1844 - categorical_accuracy: 0.2005 - val_loss: 1.6163 - val_categorical_accuracy: 0.1708
Epoch 4/2000
 - 0s - loss: 2.2235 - categorical_accuracy: 0.2010 - val_loss: 1.6186 - val_categorical_accuracy: 0.1708
Epoch 5/2000
 - 0s - loss: 2.1008 - categorical_accuracy: 0.2255 - val_loss: 1.6198 - val_categorical_accuracy: 0.1708
Epoch 6/2000
 - 0s - loss: 2.1400 - categorical_accuracy: 0.2302 - val_loss: 1.6183 - val_categorical_accuracy: 0.1708
Epoch 7/2000
 - 0s - loss: 2.1920 - categorical_accuracy: 0.1990 - val_loss: 1.6173 - val_categorical_accuracy: 0.1708
Epoch 8/2000
 - 0s - loss: 2.0913 - categorical_accuracy: 0.2250 - val_loss: 1.6151 - val_categorical_accuracy: 0.1708


 - 0s - loss: 1.8314 - categorical_accuracy: 0.2953 - val_loss: 1.5084 - val_categorical_accuracy: 0.3208
Epoch 68/2000
 - 0s - loss: 1.8383 - categorical_accuracy: 0.2802 - val_loss: 1.5058 - val_categorical_accuracy: 0.3229
Epoch 69/2000
 - 0s - loss: 1.8119 - categorical_accuracy: 0.2802 - val_loss: 1.5064 - val_categorical_accuracy: 0.3271
Epoch 70/2000
 - 0s - loss: 1.8311 - categorical_accuracy: 0.2995 - val_loss: 1.5069 - val_categorical_accuracy: 0.3229
Epoch 71/2000
 - 0s - loss: 1.7632 - categorical_accuracy: 0.3109 - val_loss: 1.5084 - val_categorical_accuracy: 0.3167
Epoch 72/2000
 - 0s - loss: 1.8118 - categorical_accuracy: 0.2948 - val_loss: 1.5070 - val_categorical_accuracy: 0.3229
Epoch 73/2000
 - 0s - loss: 1.8446 - categorical_accuracy: 0.2849 - val_loss: 1.5073 - val_categorical_accuracy: 0.3229
Epoch 74/2000
 - 0s - loss: 1.8370 - categorical_accuracy: 0.2854 - val_loss: 1.5060 - val_categorical_accuracy: 0.3229
Epoch 75/2000
 - 0s - loss: 1.8308 - categorical_accur

 - 0s - loss: 1.6205 - categorical_accuracy: 0.3734 - val_loss: 1.4838 - val_categorical_accuracy: 0.3063
Epoch 136/2000
 - 0s - loss: 1.6322 - categorical_accuracy: 0.3698 - val_loss: 1.4822 - val_categorical_accuracy: 0.3042
Epoch 137/2000
 - 0s - loss: 1.6531 - categorical_accuracy: 0.3495 - val_loss: 1.4792 - val_categorical_accuracy: 0.3104
Epoch 138/2000
 - 0s - loss: 1.6650 - categorical_accuracy: 0.3396 - val_loss: 1.4757 - val_categorical_accuracy: 0.3104
Epoch 139/2000
 - 0s - loss: 1.6758 - categorical_accuracy: 0.3510 - val_loss: 1.4730 - val_categorical_accuracy: 0.3167
Epoch 140/2000
 - 0s - loss: 1.6216 - categorical_accuracy: 0.3651 - val_loss: 1.4733 - val_categorical_accuracy: 0.3125
Epoch 141/2000
 - 0s - loss: 1.6860 - categorical_accuracy: 0.3542 - val_loss: 1.4734 - val_categorical_accuracy: 0.3229
Epoch 142/2000
 - 0s - loss: 1.6683 - categorical_accuracy: 0.3604 - val_loss: 1.4722 - val_categorical_accuracy: 0.3187
Epoch 143/2000
 - 0s - loss: 1.6585 - categoric

Epoch 203/2000
 - 0s - loss: 1.6060 - categorical_accuracy: 0.3906 - val_loss: 1.4406 - val_categorical_accuracy: 0.3500
Epoch 204/2000
 - 0s - loss: 1.5512 - categorical_accuracy: 0.3932 - val_loss: 1.4421 - val_categorical_accuracy: 0.3500
Epoch 205/2000
 - 0s - loss: 1.5736 - categorical_accuracy: 0.3974 - val_loss: 1.4407 - val_categorical_accuracy: 0.3479
Epoch 206/2000
 - 0s - loss: 1.5617 - categorical_accuracy: 0.3917 - val_loss: 1.4362 - val_categorical_accuracy: 0.3500
Epoch 207/2000
 - 0s - loss: 1.5730 - categorical_accuracy: 0.3807 - val_loss: 1.4363 - val_categorical_accuracy: 0.3479
Epoch 208/2000
 - 0s - loss: 1.5460 - categorical_accuracy: 0.3932 - val_loss: 1.4374 - val_categorical_accuracy: 0.3479
Epoch 209/2000
 - 0s - loss: 1.5761 - categorical_accuracy: 0.3745 - val_loss: 1.4339 - val_categorical_accuracy: 0.3500
Epoch 210/2000
 - 0s - loss: 1.5131 - categorical_accuracy: 0.4000 - val_loss: 1.4329 - val_categorical_accuracy: 0.3521
Epoch 211/2000
 - 0s - loss: 1.5

Epoch 271/2000
 - 0s - loss: 1.5021 - categorical_accuracy: 0.4187 - val_loss: 1.3998 - val_categorical_accuracy: 0.3604
Epoch 272/2000
 - 0s - loss: 1.5255 - categorical_accuracy: 0.4109 - val_loss: 1.3962 - val_categorical_accuracy: 0.3604
Epoch 273/2000
 - 0s - loss: 1.4885 - categorical_accuracy: 0.4099 - val_loss: 1.3989 - val_categorical_accuracy: 0.3625
Epoch 274/2000
 - 0s - loss: 1.4955 - categorical_accuracy: 0.4208 - val_loss: 1.3984 - val_categorical_accuracy: 0.3625
Epoch 275/2000
 - 0s - loss: 1.4752 - categorical_accuracy: 0.4141 - val_loss: 1.3977 - val_categorical_accuracy: 0.3604
Epoch 276/2000
 - 0s - loss: 1.5084 - categorical_accuracy: 0.4000 - val_loss: 1.3965 - val_categorical_accuracy: 0.3625
Epoch 277/2000
 - 0s - loss: 1.4633 - categorical_accuracy: 0.4271 - val_loss: 1.3971 - val_categorical_accuracy: 0.3646
Epoch 278/2000
 - 0s - loss: 1.4676 - categorical_accuracy: 0.4245 - val_loss: 1.3953 - val_categorical_accuracy: 0.3625
Epoch 279/2000
 - 0s - loss: 1.4

...

Epoch 1697/2000
 - 0s - loss: 0.9611 - categorical_accuracy: 0.5823 - val_loss: 1.0549 - val_categorical_accuracy: 0.5167
Epoch 1698/2000
 - 0s - loss: 0.9826 - categorical_accuracy: 0.5526 - val_loss: 1.0577 - val_categorical_accuracy: 0.5167
Epoch 1699/2000
 - 0s - loss: 0.9967 - categorical_accuracy: 0.5771 - val_loss: 1.0580 - val_categorical_accuracy: 0.5188
Epoch 1700/2000
 - 0s - loss: 0.9816 - categorical_accuracy: 0.5500 - val_loss: 1.0558 - val_categorical_accuracy: 0.5188
Epoch 1701/2000
 - 0s - loss: 0.9658 - categorical_accuracy: 0.5781 - val_loss: 1.0578 - val_categorical_accuracy: 0.5250
Epoch 1702/2000
 - 0s - loss: 1.0038 - categorical_accuracy: 0.5620 - val_loss: 1.0572 - val_categorical_accuracy: 0.5208
Epoch 1703/2000
 - 0s - loss: 0.9666 - categorical_accuracy: 0.5729 - val_loss: 1.0550 - val_categorical_accuracy: 0.5250
Epoch 1704/2000
 - 0s - loss: 0.9739 - categorical_accuracy: 0.5708 - val_loss: 1.0580 - val_categorical_accuracy: 0.5208
Epoch 1705/2000
 - 0s - 

Epoch 1765/2000
 - 0s - loss: 0.9848 - categorical_accuracy: 0.5516 - val_loss: 1.0520 - val_categorical_accuracy: 0.5271
Epoch 1766/2000
 - 0s - loss: 0.9921 - categorical_accuracy: 0.5745 - val_loss: 1.0526 - val_categorical_accuracy: 0.5250
Epoch 1767/2000
 - 0s - loss: 1.0006 - categorical_accuracy: 0.5688 - val_loss: 1.0522 - val_categorical_accuracy: 0.5250
Epoch 1768/2000
 - 0s - loss: 0.9914 - categorical_accuracy: 0.5510 - val_loss: 1.0545 - val_categorical_accuracy: 0.5229
Epoch 1769/2000
 - 0s - loss: 0.9730 - categorical_accuracy: 0.5589 - val_loss: 1.0571 - val_categorical_accuracy: 0.5188
Epoch 1770/2000
 - 0s - loss: 0.9752 - categorical_accuracy: 0.5677 - val_loss: 1.0566 - val_categorical_accuracy: 0.5188
Epoch 1771/2000
 - 0s - loss: 0.9918 - categorical_accuracy: 0.5755 - val_loss: 1.0563 - val_categorical_accuracy: 0.5229
Epoch 1772/2000
 - 1s - loss: 0.9899 - categorical_accuracy: 0.5740 - val_loss: 1.0569 - val_categorical_accuracy: 0.5250
Epoch 1773/2000
 - 1s - 

Epoch 1832/2000
 - 0s - loss: 0.9675 - categorical_accuracy: 0.5646 - val_loss: 1.0543 - val_categorical_accuracy: 0.5208
Epoch 1833/2000
 - 0s - loss: 0.9845 - categorical_accuracy: 0.5656 - val_loss: 1.0565 - val_categorical_accuracy: 0.5146
Epoch 1834/2000
 - 0s - loss: 0.9362 - categorical_accuracy: 0.5771 - val_loss: 1.0581 - val_categorical_accuracy: 0.5208
Epoch 1835/2000
 - 0s - loss: 0.9929 - categorical_accuracy: 0.5620 - val_loss: 1.0531 - val_categorical_accuracy: 0.5208
Epoch 1836/2000
 - 0s - loss: 1.0058 - categorical_accuracy: 0.5594 - val_loss: 1.0521 - val_categorical_accuracy: 0.5188
Epoch 1837/2000
 - 0s - loss: 0.9763 - categorical_accuracy: 0.5797 - val_loss: 1.0541 - val_categorical_accuracy: 0.5208
Epoch 1838/2000
 - 0s - loss: 0.9874 - categorical_accuracy: 0.5583 - val_loss: 1.0534 - val_categorical_accuracy: 0.5208
Epoch 1839/2000
 - 0s - loss: 0.9999 - categorical_accuracy: 0.5625 - val_loss: 1.0549 - val_categorical_accuracy: 0.5188
Epoch 1840/2000
 - 0s - 

Epoch 1899/2000
 - 0s - loss: 0.9814 - categorical_accuracy: 0.5635 - val_loss: 1.0542 - val_categorical_accuracy: 0.5208
Epoch 1900/2000
 - 0s - loss: 0.9996 - categorical_accuracy: 0.5604 - val_loss: 1.0525 - val_categorical_accuracy: 0.5250
Epoch 1901/2000
 - 0s - loss: 0.9776 - categorical_accuracy: 0.5646 - val_loss: 1.0525 - val_categorical_accuracy: 0.5229
Epoch 1902/2000
 - 0s - loss: 0.9971 - categorical_accuracy: 0.5708 - val_loss: 1.0515 - val_categorical_accuracy: 0.5229
Epoch 1903/2000
 - 0s - loss: 0.9622 - categorical_accuracy: 0.5677 - val_loss: 1.0509 - val_categorical_accuracy: 0.5188
Epoch 1904/2000
 - 0s - loss: 0.9965 - categorical_accuracy: 0.5651 - val_loss: 1.0505 - val_categorical_accuracy: 0.5188
Epoch 1905/2000
 - 0s - loss: 0.9747 - categorical_accuracy: 0.5724 - val_loss: 1.0509 - val_categorical_accuracy: 0.5188
Epoch 1906/2000
 - 0s - loss: 0.9795 - categorical_accuracy: 0.5510 - val_loss: 1.0501 - val_categorical_accuracy: 0.5250
Epoch 1907/2000
 - 0s - 

Epoch 1966/2000
 - 0s - loss: 0.9771 - categorical_accuracy: 0.5745 - val_loss: 1.0546 - val_categorical_accuracy: 0.5250
Epoch 1967/2000
 - 0s - loss: 0.9834 - categorical_accuracy: 0.5755 - val_loss: 1.0554 - val_categorical_accuracy: 0.5250
Epoch 1968/2000
 - 0s - loss: 0.9692 - categorical_accuracy: 0.5917 - val_loss: 1.0567 - val_categorical_accuracy: 0.5250
Epoch 1969/2000
 - 0s - loss: 1.0131 - categorical_accuracy: 0.5682 - val_loss: 1.0561 - val_categorical_accuracy: 0.5271
Epoch 1970/2000
 - 0s - loss: 0.9634 - categorical_accuracy: 0.5708 - val_loss: 1.0571 - val_categorical_accuracy: 0.5250
Epoch 1971/2000
 - 0s - loss: 0.9790 - categorical_accuracy: 0.5615 - val_loss: 1.0525 - val_categorical_accuracy: 0.5250
Epoch 1972/2000
 - 0s - loss: 1.0054 - categorical_accuracy: 0.5495 - val_loss: 1.0554 - val_categorical_accuracy: 0.5208
Epoch 1973/2000
 - 0s - loss: 0.9793 - categorical_accuracy: 0.5729 - val_loss: 1.0520 - val_categorical_accuracy: 0.5250
Epoch 1974/2000
 - 0s - 

In [ ]:
scores = model.evaluate(X_train, Y_train, batch_size=30)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
print(X_test[:5])
predictions = model.predict_classes(X_test[:5], verbose = 1) + 1
print()
print(predictions)

In [ ]:
print(X_test[:1])
predictions = model.predict_classes(X_test[:1], verbose = 1) + 1
print()
print(predictions)

In [ ]:
import time
i = 0

def hello(model):
    global i
    print("read csv, Interval {}".format(i))
    predictions = model.predict(X_test[:1], verbose = 1)
    print(predictions)
    i = i + 1
    
nexttime = time.time()
while True:
    hello(model)          # take t sec
    nexttime += 5
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)

### Comparing the effects of training the model on various subsamples of the data

In [ ]:
result_dict = {}
subset_data_list = [0.2, 0.4, 0.6, 0.8]

for subset_data in subset_data_list:
    #training the model on a proportion of data and save the loss and accuracy
    
    model = create_model()
    model.compile(optimizer = Adam(lr = 0.000005), loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
    
    #adding callbacks
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.5,
                                  patience=50, min_lr=1e-7, verbose = 1)
    history = History()
    logger = CSVLogger(os.path.join(rootPath, 'logs', 'log_results-{}.csv'.format(subset_data)))
    
    #training the model with subset of data
    history = model.fit(X_train, Y_train,
                        validation_split = 1 - subset_data,
                        epochs = 1000,
                        batch_size = 64,
                        verbose = 2,
                        callbacks = [reduce_lr, history, logger])
    
    result_dict[subset_data] = {
        'training_loss' : history.history['loss'],
        'validation_loss': history.history['val_loss'],
        'training_accuracy': history.history['categorical_accuracy'],
        'validation_accuracy': history.history['val_categorical_accuracy']
    }

In [ ]:
def plotGraph(result_dict, plot_var):
    #this plots a graph of training loss vs epochs
    for subset_data in result_dict.keys():
        ax = sns.lineplot(x = np.arange(1000), 
                          y = result_dict[subset_data][plot_var], label =subset_data)
        ax.set(xlabel = 'training epoch', ylabel = plot_var)
        
    if plot_var == "training_loss":
        #rescale for easier comparison
        ax.set_ylim(1, 2.5)

        
    ax.set_title('{} with epoch'.format(plot_var))
    ax.legend()
    
    return ax

In [ ]:
plotGraph(result_dict, 'training_loss')

In [ ]:
plotGraph(result_dict,'validation_loss')

In [ ]:
plotGraph(result_dict,'training_accuracy')

In [ ]:
plotGraph(result_dict,'validation_accuracy')

In [ ]:
predictions = model.predict(X_test[:5], verbose = 1)
# predictions = model.predict_classes(X_test[:50], verbose=1)

In [ ]:
print(X_test[:5])

In [ ]:
print(predictions)